In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

from functions import *

In [2]:
X = pd.read_csv("indep_x.csv", index_col = 0)

In [3]:
check_balance(X, "treated", X.drop(["treated"], axis = 1).columns)

c:\Users\Saverio\anaconda3\lib\site-packages\scipy\stats\_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


,Covariate,Test,p-value,Significant,Difference,Standardized Difference
0,age,Welch's t-test,0.410959,False,-0.159262,-0.133256
1,gender,Welch's t-test,0.570504,False,-0.046138,-0.091930
2,scholarship,Welch's t-test,0.340511,False,0.040157,0.154775
3,1st_year,Welch's t-test,0.052406,False,0.077580,0.316336
4,gpa,Welch's t-test,0.799217,False,0.118934,0.041180
5,1st_time,Welch's t-test,0.083237,False,0.038961,0.282892
6,taste,Welch's t-test,0.019981,True,0.239747,0.380138
7,importance,Welch's t-test,0.090261,False,0.182160,0.275559
8,expected_grade,Welch's t-test,0.200697,False,0.534689,0.207866
9,knowledge,Welch's t-test,0.269851,False,0.082365,0.179123


In [4]:
balance_data = X.copy() #i'm making a copy of the dataset

In [5]:
X_notreated = X.copy()
X_notreated = X_notreated.drop(["treated"], axis = 1)

In [6]:
results = [] #initialize an empty list in wich the result for each variable will be stored

In [7]:
# We run a regression for each variable: Variable ~ Treatment
for col in X_notreated.columns:
    y = balance_data[col]
    X = sm.add_constant(balance_data["treated"])  # adds intercept
    model = sm.OLS(y, X).fit(cov_type="HC3")
    fval = model.f_pvalue #F-statistic
    results.append((col, fval))

In [8]:
balance_test_df = pd.DataFrame(results, columns=["Variable", "F-value"])
balance_test_df.sort_values("F-value", ascending=False, inplace=True)

In [9]:
balance_test_df

,Variable,F-value
4,gpa,0.800495
1,gender,0.573031
0,age,0.413999
2,scholarship,0.343570
9,knowledge,0.272979
8,expected_grade,0.203635
7,importance,0.092290
5,1st_time,0.083218
3,1st_year,0.053363
6,taste,0.020761
